In [1]:
import pandas as pd
import json

data = pd.read_csv("https://github.com/ga-students/DAT-NYC-37/blob/master/lessons/lesson-13/assets/dataset/stumbleupon.tsv?raw=true", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


## Predicting "Greenness" Of Content

This dataset comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender.  

A description of the columns is below

#### What are 'evergreen' sites?

Evergreen sites are those that are always relevant.  As opposed to breaking news or current events, evergreen websites are relevant no matter the time or season. 

*A sample of URLs is below, where `label = 1` are 'evergreen' websites*

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

---

### Objective: Predict if a given site will be evergreen based on the above features

**Problem:** Some of the above features are text-only (`title`, `url`, `body`). How can I leverage the modeling techniques we've covered so far to utilize text based features?

**Solution:** Transform text features into many numerical features.
  - Count Vectorization
  - Term frequency/inverse document frequency (TF-IDF) Vectorization.
---

In [11]:
# Q: Add a feature that is True if a Title contains recipe. Do such articles have a relationship with 'evergrenness'?

data['title'].str.lower().str.contains('recipe')



0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17       None
18      False
19      False
20      False
21       True
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
7365    False
7366    False
7367    False
7368    False
7369    False
7370    False
7371     True
7372    False
7373    False
7374    False
7375    False
7376     True
7377    False
7378     True
7379    False
7380     True
7381    False
7382    False
7383    False
7384    False
7385    False
7386    False
7387    False
7388    False
7389    False
7390    False
7391    False
7392    False
7393    False
7394    False
Name: title, dtype: object

### A primitive solution:

Maybe certain keywords in the title are predictive for whether a website is evergreen or not? For instance, maybe recipes are usually "evergreen"?

We could create a feature for the title containing 'recipe'. Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

In [12]:
# Option 1: Create a function to check for this

def has_recipe(text_in):
    try:
        if 'recipe' in str(text_in).lower():
            return 1
        else:
            return 0
    except: 
        return 0
        
# .map applies a function to each row in our series and returns the result as a list.
data['recipe'] = data['title'].map(has_recipe)

# Note: We can also use lambda functions, instead of the above function definition
# data['recipe'] = data['title'].map(lambda t: 1 if 'recipe' in str(t).lower() else 0)

# Option 3: string functions
data['recipe'] = data['title'].str.lower().str.contains('recipe')

pd.crosstab(data['recipe'], data['label'])

label,0,1
recipe,,
False,3510,2939
True,82,852


Recipe articles seem to be evergreen more often than not -- but obviously most of our articles are not recipes. How can we apply this more generally?

**CountVectorizer:** converts a collection of text into a matrix of features.  Each row will be a sample (an article or piece of text) and each column will be a text feature (usually a count or binary feature per word).

### Demo: Understanding Count Vectorization

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

titles = [
    "IBM Sees Electronic Calls Air Breathing Batteries",
    "The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races"
    "The Chicago Bulls won"
]

vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
vectorized_titles = vectorizer.fit_transform(titles)

print "Feature names: \n", vectorizer.get_feature_names()
print "Feature counts: \n", vectorized_titles.todense()
print

# Represent Count Vectorized results as a dataframe so we can preview it more easily.
pd.DataFrame(
    columns=vectorizer.get_feature_names(),
    index=['Article1', 'Article2'],
    data=vectorized_titles.todense()
)

Feature names: 
[u'advantages', u'advantages racesthe', u'air', u'air breathing', u'batteries', u'breathing', u'breathing batteries', u'bulls', u'bulls won', u'calls', u'calls air', u'chicago', u'chicago bulls', u'electronic', u'electronic calls', u'electronic futuristic', u'eliminates', u'eliminates advantages', u'fully', u'fully electronic', u'futuristic', u'futuristic starting', u'gun', u'gun eliminates', u'ibm', u'ibm sees', u'racesthe', u'racesthe chicago', u'sees', u'sees electronic', u'starting', u'starting gun', u'won']
Feature counts: 
[[0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0]
 [1 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1]]



,advantages,advantages racesthe,air,air breathing,batteries,breathing,breathing batteries,bulls,bulls won,calls,...,gun eliminates,ibm,ibm sees,racesthe,racesthe chicago,sees,sees electronic,starting,starting gun,won
Article1,0,0,1,1,1,1,1,0,0,1,...,0,1,1,0,0,1,1,0,0,0
Article2,1,1,0,0,0,0,0,1,1,0,...,1,0,0,1,1,0,0,1,1,1


In [17]:
# TODO: Apply count vectorization to all titles

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
vectorized_titles = vectorizer.fit_transform(data['title'].dropna())

print "Feature names: \n", vectorizer.get_feature_names()
# print "Feature counts: \n", vectorized_titles.todense()
# print

vectorized_titles

len(vectorizer.get_feature_names())

print "Feature names: \n", vectorizer.get_feature_names()

Feature names: 
[u'00', u'000', u'000f', u'007', u'01', u'021', u'024', u'04', u'09', u'0db', u'10', u'100', u'100m', u'100th', u'101', u'106', u'11', u'110', u'1116', u'117', u'11th', u'12', u'120', u'1258', u'1271771011252', u'128', u'129', u'13', u'130th', u'132', u'14', u'144190', u'1460', u'149', u'14th', u'15', u'150', u'150lb', u'1549', u'155', u'1581', u'16', u'160', u'163', u'1644', u'16748703', u'16gb', u'17', u'170', u'1785', u'17a', u'18', u'180', u'1830s', u'18in', u'19', u'1908', u'1920s', u'1940s', u'1944', u'1948', u'1949', u'1950', u'1950s', u'1954', u'1960', u'1960s', u'1963', u'1969', u'1970', u'1970s', u'1972', u'1977', u'198', u'1985', u'1990', u'1994', u'1995', u'1996', u'1998', u'1999', u'19th', u'1st', u'1w', u'20', u'200', u'2000', u'2000s', u'2002', u'2003', u'2005', u'2006', u'2007', u'2008', u'2009', u'2009s', u'2010', u'2011', u'20110131102815', u'2012', u'2013', u'2017', u'2020', u'2030', u'206', u'2068', u'208mph', u'209', u'20th', u'21', u'21453248', u'2

## Demo: Term-frequency, Inverse document frequency (Tf-Idf)

An alternative bag-of-words approach to CountVectorizer is a Term Frequency - Inverse Document Frequency (TF-IDF) representation.

TF-IDF uses the product of two intermediate values, the **Term Frequency** and **Inverse Document Frequency**.

- **Term Frequency** is equivalent to CountVectorizer features, just the number of times a word appears in the document (i.e. count).

- **Document Frequency** is the percentage of documents that a particular word appears in. 

For example, “the” would be 100% while “Syria” is much lower.  

Inverse Document Frequency is just 1/Document Frequency.


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

titles = [
    "IBM Sees Electronic Calls Air Breathing Batteries",
    "The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races"
]

vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf = False)
vectorized_titles = vectorizer.fit_transform(titles)

print "Feature names: \n", vectorizer.get_feature_names()
print "Feature counts: \n", vectorized_titles.todense()
print "IDF weights", vectorizer.idf_


# Represent Count Vectorized results as a dataframe so we can preview it more easily.
pd.DataFrame(
    columns=vectorizer.get_feature_names(),
    index=['Article1', 'Article2'],
    data=vectorized_titles.todense()
)

Feature names: 
[u'advantages', u'air', u'batteries', u'breathing', u'calls', u'electronic', u'eliminates', u'fully', u'futuristic', u'gun', u'ibm', u'races', u'sees', u'starting']
Feature counts: 
[[ 0.          0.39204401  0.39204401  0.39204401  0.39204401  0.27894255
   0.          0.          0.          0.          0.39204401  0.
   0.39204401  0.        ]
 [ 0.36499647  0.          0.          0.          0.          0.25969799
   0.36499647  0.36499647  0.36499647  0.36499647  0.          0.36499647
   0.          0.36499647]]
IDF weights [ 1.40546511  1.40546511  1.40546511  1.40546511  1.40546511  1.
  1.40546511  1.40546511  1.40546511  1.40546511  1.40546511  1.40546511
  1.40546511  1.40546511]


,advantages,air,batteries,breathing,calls,electronic,eliminates,fully,futuristic,gun,ibm,races,sees,starting
Article1,0.000000,0.392044,0.392044,0.392044,0.392044,0.278943,0.000000,0.000000,0.000000,0.000000,0.392044,0.000000,0.392044,0.000000
Article2,0.364996,0.000000,0.000000,0.000000,0.000000,0.259698,0.364996,0.364996,0.364996,0.364996,0.000000,0.364996,0.000000,0.364996


In [34]:
# TODO: Determine Tf-Idf of title 
# Find the words with 100 highest and lowest inverse document frequency

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf = False)

vectorizer.fit(data['title'].dropna())
vectorized_titles = vectorizer.transform(titles)

feature_names         = vectorizer.get_feature_names()  # Returns all feature names
inverse_document_freq = vectorizer.idf_  # Returns document frequencies

tfidf = pd.DataFrame(
    {
        "feature_names": feature_names,
        "inverse_document_freq": inverse_document_freq
    }
)

sorted_tfidf = tfidf.sort_values(by='inverse_document_freq')

sorted_tfidf.tail(100)
sorted_tfidf.head(100)

,feature_names,inverse_document_freq
7510,recipe,3.542320
2015,com,3.556185
7513,recipes,3.920619
3623,food,4.196644
1809,chocolate,4.342550
8616,sports,4.354111
9805,video,4.477725
6292,news,4.517999
1011,best,4.584061
3375,fashion,4.714114


In [ ]:
## TODO: Determine Tf-Idf of title 
# Find the words with 100 highest and lowest inverse document frequency

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf = False)

vectorizer.fit(data['body'].dropna())
vectorized_titles = vectorizer.transform(titles)

feature_names         = vectorizer.get_feature_names()  # Returns all feature names
inverse_document_freq = vectorizer.idf_  # Returns document frequencies

tfidf = pd.DataFrame(
    {
        "feature_names": feature_names,
        "inverse_document_freq": inverse_document_freq
    }
)

sorted_tfidf = tfidf.sort_values(by='inverse_document_freq')

sorted_tfidf.head(100)
sorted_tfidf.tail(100)

In [41]:
vectorized_titles

len()

<2x85212 sparse matrix of type '<type 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

 ### Demo: Use of the Count Vectorizer with ngrams
 
 We can use the `ngram_range` parameter to find ngrams -- groups of n words.

In [110]:
# Note the inclusion of ngram_range
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
vectorized_titles = vectorizer.fit_transform(titles)

print "Feature names: \n", vectorizer.get_feature_names()
print "Feature counts: \n", vectorized_titles.todense()
print

# Represent Count Vectorized results as a dataframe so we can preview it more easily.
pd.DataFrame(
    columns=vectorizer.get_feature_names(),
    index=['Article1', 'Article2'],
    data=vectorized_titles.todense()
)

Feature names: 
[u'advantages', u'advantages races', u'air', u'air breathing', u'batteries', u'breathing', u'breathing batteries', u'calls', u'calls air', u'electronic', u'electronic calls', u'electronic futuristic', u'eliminates', u'eliminates advantages', u'fully', u'fully electronic', u'futuristic', u'futuristic starting', u'gun', u'gun eliminates', u'ibm', u'ibm sees', u'races', u'sees', u'sees electronic', u'starting', u'starting gun']
Feature counts: 
[[0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0]
 [1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1]]



,advantages,advantages races,air,air breathing,batteries,breathing,breathing batteries,calls,calls air,electronic,...,futuristic starting,gun,gun eliminates,ibm,ibm sees,races,sees,sees electronic,starting,starting gun
Article1,0,0,1,1,1,1,1,1,1,1,...,0,0,0,1,1,0,1,1,0,0
Article2,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,1,1


 ### Demo: Build a random forest model to predict evergreeness of a website using the title features

In [29]:
from sklearn.ensemble import RandomForestClassifier

titles = data['title'].fillna('')

model = RandomForestClassifier(n_estimators = 20)
    
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles).toarray()
y = data['label']

from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.81608807  0.82887978  0.82386112], Average AUC 0.822942990477


### Exercise: Build a random forest model to predict evergreeness of a website using the title features and quantitative features

In [ ]:
## TODO

 ### Exercise: Build a random forest model to predict evergreeness of a website using the body features

In [ ]:
## TODO

 ### Exercise: Use `TfIdfVectorizer` instead of `CountVectorizer` - is this an improvement?

In [ ]:
## TODO